<a href="https://colab.research.google.com/github/jinyingtld/python/blob/main/MMDetection_Faster_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [MMDetection Faster R-CNN 源码详解（一)](https://zhuanlan.zhihu.com/p/166248079)

本系列文章会详细剖析 MMDetection 是如何实现 Faster R-CNN 的，在本篇文章（一）中会详细的讲解 Faster R-CNN 中 backbone 相关的代码。

下图是 MMDetection 实现的 Faster R-CNN 的结果。R-50 代表的是 ResNet 50，X-101 代表的是 ResNeXt 101。所以在本篇文章中会详细的讲解这两个 backbone 的网络结构和在 MMDetection 中的源码实现。

![Image](https://pic3.zhimg.com/80/v2-a5c9e0fe5d3f2ff43e82a570eb813326_720w.jpg)


## 一、ResNet

![](https://pic4.zhimg.com/80/v2-1155f81ae526e777e575900f3e84b65b_720w.jpg)

上图是 ResNet 的网络结构，我们来详细的分析一下。

网络的输入是 224×224 的图片，首先会经过 stem 模块。stem 模块对于不同深度的 ResNet 使用的都是相同的结构，都会经过一个conv 7×7（channels：64，stride：2，padding：3） 的卷积。输出特征图的大小为 112×112。然后再次下采样，经过 Max Pooling 3×3（stride：2，padding：1） 得到 56 ×56 的特征图。当经过了 stem 模块，会完成 4 倍的下采样。(x: 224X224X3 -->conv:( 7x7 stride=2,padding=3 output=112x112)-->BN--->ReLu-->Maxpool(3x3, stride=2 padding=1 downsampling out=112/2=56)

接下来会进入 4 组堆叠的残差模块，除了经过第一组残差模块特征图大小以及通道数不发生变化以外，后面三组残差模块，每经过一组，特征图的大小都会缩小一半，通道数扩张为原来 2 倍。经过四组残差模块进行特征提取后，特征图的大小变为 7×7，然后对这 7×7 的特征图做全局均值池化(avg Pooling)。最后接上全连接层，将通道数变为类别个数进行输出。

在 ResNet 中最主要的结构就是残差模块，我们下面就详细解释一下残差模块的结构。

残差模块含有shortcut连接, 会将经过卷积操作的结果和未经过卷积操作的结果相加. (注意: 注意：这里是相加不是拼接）对于相加后的结果再用激活函数计算. 不同的深度的 ResNet，使用了不同的残差模块。深度小于50的ResNet, 使用BasicBlock, 深度大于等于50的ResNet使用Bottleneck. 下面我们就来分别看一下这两种结构。

##（一）BasicBlock（深度小于 50 的 ResNet 使用）
BasicBlock 用于深度小于 50 的 ResNet（ResNet 18、34）。它的结构如下图：
![](https://pic4.zhimg.com/80/v2-8ef93910b9b1d74f45cef442424b31fb_720w.jpg)
图三：BasicBlock 结构

对于每个BasicBlock, 有两个分支. 一个分支用来正常的卷积,另一个分支用于跳跃连接. 输入一张特征图后会经过两个3x3的卷积提取特征,然后将卷积操作后的特征图与输入的特征图相加，再用 ReLU 激活函数进行计算。在 BasicBlock 中，卷积的通道数不发生改变，可以直接将输入的特征图和卷积后的特征图相加。

上面的操作既不会改变特征图的大小，又不会改变特征图的通道数，那么我们如何进行下采样呢？换句话说，也就是上面的情况是针对同一个 stage 之内的 BasicBlock，对于不同 stage 之间的 BasicBlock 我们怎么处理呢（如：conv2_2 和 conv3_1）？

我们对每个stage中的第一个block进行更改.将第一个3x3的卷积的步长变为原来的2倍, 通过数也扩大为原来的2倍,padding不变还为1. 这样的化经过卷积后的特征图的大小变为原来的1/2,通道数会扩张为原来的2倍. 对于shortcut分支,使用步长为2的 1×1 卷积用于下采样和扩张通道数，这样特征图的大小变成原来的 1/2，通道数也会扩张为原来的 2 倍。和卷积分支的输出特征图形状相同。然后我们将两个分支的结果相加，再通过 ReLU 激活函数即可。如下图：

![](https://pic2.zhimg.com/v2-e849a9d912716e1c213fb5db3bb76a45_r.jpg)图四：stage 和 stage 之间的 BasicBlock

当然因为 stem 和第一个 stage 之间的通道数都是 64，且第一个 stage 不需要进行下采样。所以，我们使用图三的 block 即可。]

我们来看一下源码：


In [ ]:
import torch.nn as nn
import torch.utils.checkpoint as cp
from mmcv.cnn import (build_conv_layer, build_norm_layer, build_plugin_layer, constant_init, kaiming_init)
from mmcv.runner import load_checkpoint
from torch.nn.modules.batchnorm import _BatchNorm

from mmdet.utils import get_root_logger
from ..builder import BACKBONES
from ..utils import ResLayer

class BasicBlock(nn.Module):
    """ResNet 18, 34 使用的block"""
    # 输出通道为输入通道的倍数.(输出通道数 == 输入通道数)
    expansion = 1

    def __init__(self,
                 inplanes,
                 planes,
                 stride=1,
                 dilation=1,
                 downsample=None
                 style='pytorch',
                 with_cp=False,
                 conf_cfg=None,
                 norm_cfg=dict(type='BN'),
                 dcn=None,
                 plugins=None):
        super(BasicBlock, self).__init__()
        assert dcn is None, 'Not implemented yet.'
        assert plugins is None, 'Not implemented yet.'
        # conv3x3 ---> bn1 --> relu --> conv3x3 ---> bn2 --> relu

        #bn1, bn2
        self.norm1_name, norm1 = build_norm_layer(norm_cfg, planes, postfix=1)
        self.norm2_name, norm2 = build_norm_layer(norm_cfg, planes, postfix=2)

        # 当conv1: 3x3 conv 
        # 当conv 为3x3 且padding = dilation时, 原特征图大小只和 stride 有关
        self.conv1 = build_conv_layer(
            conv_cfg,
            inplanes,
            planes,
            3,
            stride=stride,
            padding=dilation,
            dilation=dilation,
            bias=False)
        self.add_module(self.norm1_name,norm1)

        # conv2: 3x3 conv, stride = 1, padding = 1.
        self.conv2 = build_conv_layer(
            conv_cfg, planes, planes, 3, padding=1, bias=False)
        self.add_module(self.norm2_name, norm2)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation
        self.with_cp = with_cp

    @property
    def norm1(self):
        """nn.Module: normalization layer after the first convolution layer"""
        return getattr(self, self.norm1_name)
    
    @property
    def norm2(self):
        """nn.Module: normalization layer after the second convolution layer"""
        return getattr(self, self.norm2_name)

    def forward(self, x):
        """Forward function."""
        def _inner_forward(x):
            identity = x

            out = self.conv1(x)
            out = self.norm1(out)
            out = self.relu(out)

            out = self.conv2(out)
            out = self.norm2(out)

            # 需要保证identity 和 x的宽度相同.
            if self.downsample is not None:
                identity = self.downsample(x)
            
            # h(x) = f(x) + x
            out += identity

            return out

        # 加载预训练模型并且需要求导的时候, 使用checkpoint用时间换取空间.
        # 这是因为加载权重后可以训练的epoch数少, 可以考虑时间换取空间.
        if self.with_cp and x.requires_grad:
            # 使用 checkpoint 不保存中间计算的激活值, 在反向传播中重新计算一次中间激活值。
            # 即重新运行一次检查点部分的前向传播，这是一种以时间换空间（显存）的方法。
            out = cp.checkpoint(_inner_forward, x)
        # 不加载权重，从零开始训练。不使用 ckpt，因为训练慢。
        else:
            out = _inner_forward(x)

         # 注意：f(x) + x 之后再进行 relu
        out = self.relu(out)

        return out


## （二）Bottleneck（深度大于等于 50 的 ResNet 使用）

Bottleneck 用于深度大于等于50de ResNet (ResNet 50, 101, 152). 它的结构如下图：

![](https://pic2.zhimg.com/80/v2-b37d119bf676d64d30e0f36a72824999_720w.jpg)图五：Bottleneck 结构

对于 Bottleneck 来说，它会先使用 1×1 的卷积对输入的通道数进行压缩，然后再使用 3×3 的卷积进行卷积，最后使用 1×1 的卷积恢复通道数。然后将输入与卷积的结果相加并通过激活函数进行计算。

对于上面的操作，特征图的大小和输入输出的通道数不发生改变。如果需要下采样。需要将每个 stage 的第一个 Bottleneck 中的 3×3 卷积的步长和通道数设置为原来的 2 倍。对于 shortcut 分支，我们使用 1×1 步长为 2，通道数为原来 2 倍的卷积核进行下采样。如下图：

![](https://pic1.zhimg.com/80/v2-61b2c58524945a7af86b33b95c6f35ac_720w.jpg)
图六：stage 和 stage 之间的 BasicBlock

stem 和第一个 stage 之前。只需要扩大通道数不需要下采样，所以，我们只用将上图（图六）的 Bottleneck 中的 3×3 的卷积的步长设置为 1 即可。

我们来看一下源码：


In [ ]:
class Bottleneck(nn.Module):
    """ResNet 50, 101, 152 使用的 block

    Args:
        style:(str) 'pytorch 或 'caffe'.
                    如果使用 'pytorch', block 中 stride 为 2 的卷积层是 3x3 conv, stride=2
                    如果使用 'caffe',   block 中 stride 为 2 的卷积层是 1x1 conv, stride=2
    """
    # 输出通道数为输入通道数的倍数. (输出通道数 == 4 × 输入通道数)
    expansion = 4

    def __init__(self,
                 inplanes,
                 planes,
                 stride=1,
                 dilation=1,
                 downsample=None,
                 style='pytorch',
                 with_cp=False,
                 conv_cfg=None,
                 norm_cfg=dict(type='BN'),
                 dcn=None,
                 plugins=None):
        super(Bottleneck, self).__init__()
        assert style in ['pytorch', 'caffe']
        assert dcn is None or isinstance(dcn, dict)
        assert plugins is None or isinstance(plugins, list)
        if plugins is not None:
            allowed_position = ['after_conv1', 'after_conv2', 'after_conv3']
            assert all(p['position'] in allowed_position for p in plugins)

        self.inplanes = inplanes
        self.planes = planes
        self.stride = stride
        self.dilation = dilation
        self.style = style
        self.with_cp = with_cp
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg
        self.dcn = dcn
        self.with_dcn = dcn is not None
        self.plugins = plugins
        self.with_plugins = plugins is not None

        if self.with_plugins:
            # collect plugins for conv1/conv2/conv3
            self.after_conv1_plugins = [
                plugin['cfg'] for plugin in plugins
                if plugin['position'] == 'after_conv1'
            ]
            self.after_conv2_plugins = [
                plugin['cfg'] for plugin in plugins
                if plugin['position'] == 'after_conv2'
            ]
            self.after_conv3_plugins = [
                plugin['cfg'] for plugin in plugins
                if plugin['position'] == 'after_conv3'
            ]

        if self.style == 'pytorch':
            self.conv1_stride = 1
            self.conv2_stride = stride
        else:
            self.conv1_stride = stride
            self.conv2_stride = 1

        # conv1x1 --> bn1 --> relu
        # conv3x3 --> bn2 --> relu
        # conv1x1 --> bn3
        self.norm1_name, norm1 = build_norm_layer(norm_cfg, planes, postfix=1)
        self.norm2_name, norm2 = build_norm_layer(norm_cfg, planes, postfix=2)
        self.norm3_name, norm3 = build_norm_layer(
            norm_cfg, planes * self.expansion, postfix=3
        )

        self.conv1 = build_conv_layer(
            conv_cfg,
            inplanes,
            planes,
            kernel_size=1,
            stride=self.conv1_stride,
            bias=False)
        self.add_module(self.norm1_name, norm1)
        fallback_on_stride = False
        if self.with_dcn:
            fallback_on_stride = dcn.pop('fallback_on_stride', False)
        if not self.with_dcn or fallback_on_stride:
            self.conv2 = build_conv_layer(
                conv_cfg,
                planes,
                planes,
                kernel_size=3,
                stride=self.conv2_stride,
                padding=dilation,
                dilation=dilation,
                bias=False)
        else:
            assert self.conv_cfg is None, 'conv_cfg must be None for DCN'
            self.conv2 = build_conv_layer(
                dcn,
                planes,
                planes,
                kernel_size=3,
                stride=self.conv2_stride,
                padding=dilation,
                dilation=dilation,
                bias=False)

        self.add_module(self.norm2_name, norm2)
        self.conv3 = build_conv_layer(
            conv_cfg,
            planes,
            planes * self.expansion,
            kernel_size=1,
            bias=False)
        self.add_module(self.norm3_name, norm3)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample 

        if self.with_plugins:
            self.after_conv1_plugin_names = self.make_block_plugins(
                planes, self.after_conv1_plugins)
            self.after_conv2_plugin_names = self.make_block_plugins(
                planes, self.after_conv2_plugins)
            self.after_conv3_plugin_names = self.make_block_plugins(
                planes * self.expansion, self.after_conv3_plugins)

    def make_block_plugins(self, in_channels, plugins):
        """make plugins for block.

        Args:
            in_channels (int): Input channels of plugin.
            plugins (list[dict]): List of plugins cfg to build.

        Returns:
            list[str]: List of the names of plugin.
        """
        assert isinstance(plugins, list)
        plugin_names = []
        for plugin in plugins:
            plugin = plugin.copy()
            name, layer = build_plugin_layer(
                plugin,
                in_channels=in_channels,
                postfix=plugin.pop('postfix', ''))
            assert not hasattr(self, name), f'duplicate plugin {name}'
            self.add_module(name, layer)
            plugin_names.append(name)
        return plugin_names

    def forward_plugin(self, x, plugin_names):
        out = x
        for name in plugin_names:
            out = getattr(self, name)(x)
        return out

    @property
    def norm1(self):
        """nn.Module: normalization layer after the first convolution layer"""
        return getattr(self, self.norm1_name)

    @property
    def norm2(self):
        """nn.Module: normalization layer after the second convolution layer"""
        return getattr(self, self.norm2_name)

    @property
    def norm3(self):
        """nn.Module: normalization layer after the third convolution layer"""
        return getattr(self, self.norm3_name)
    
    def forward(self, x):
        """Forward function."""

        def _inner_forward(x):
            identity = x

            out = self.conv1(x)
            out = self.norm1(out)
            out = self.relu(out)

            if self.with_plugins:
                out = self.forward_plugin(out, self.after_conv1_plugin_names)

            out = self.conv2(out)
            out = self.norm2(out)
            out = self.relu(out)

            if self.with_plugins:
                out = self.forward_plugin(out, self.after_conv2_plugin_names)

            out = self.conv3(out)
            out = self.norm3(out)

            if self.with_plugins:
                out = self.forward_plugin(out, self.after_conv3_plugin_names)

            if self.downsample is not None:
                identity = self.downsample(x)

            out += identity

            return out

        if self.with_cp and x.requires_grad:
            out = cp.checkpoint(_inner_forward, x)
        else:
            out = _inner_forward(x)

        out = self.relu(out)

        return out


## 下面我们来看一下 ResNet 类的源码：

In [ ]:
class ResNet(nn.Moduel):
    """ResNet backbone.

    Args:
        depth:                     (int)   ResNet 的深度, 可以是 {18, 34, 50, 101, 152}.
        in_channels:               (int)   输入图像的通道数(默认: 3).
        stem_channels:             (int)   stem 的通道数(默认: 64).
        base_channels:             (int)   ResNet 的 res layer 的基础通道数(默认: 64).
        num_stages:                (int)   使用 ResNet 的 stage 数量(默认: 4).
        strides:         (Sequence[int])   每个 stage 的第一个 block 的 stride, 如果为 2 进行 2 倍下采样.
        dilations:       (Sequence[int])   每个 stage 中所有 block 的第一个卷积层的 dilation.
        out_indices:     (Sequence[int])   需要输出的 stage 的索引.
        style:                     (str)   'pytorch' 或 'caffe'.
                                           如果使用 'pytorch', block 中 stride 为 2 的卷积层是 3x3 conv2, stride=2
                                           如果使用 'caffe',   block 中 stride 为 2 的卷积层是 1x1 conv1, stride=2
        deep_stem:                (bool)   如果为 True, 将 stem 的 7x7 conv 替换为 3 个 3x3 conv.
        avg_down:                 (bool)   在下采样的时候使用 Avg pool 2x2 stride=2 代替带步长的卷积.
        frozen_stages:             (int)   冻结的 stage 数(停止更新梯度, 并开启eval模式), -1 代表不冻结.
        conv_cfg:                 (dict)   构建 conv 的 config.
        norm_cfg:                 (dict)   构建 norm 的 config.
        norm_eval:                (bool)   是否设置 norm 层为 eval 模式. 即冻结参数状态(mean, var).
        dcn:                      (dict)   构建 DCN 的 config.
        stage_with_dcn: (Sequence[bool])   需要使用 DCN 的 stage.
        plugins:            (list[dict])   为 stage 提供插件.
        with_cp:                  (bool)   是否加载 checkpoint. 使用 checkpoint 会节省一部分内存, 同时会减少训练时间.
        zero_init_residual:       (bool)   是否使用 0 对所有 block 中的最后一个 norm 层初始化, 使其为恒等映射.

    Example:
        >>> from mmdet.models import ResNet
        >>> import torch
        >>> self = ResNet(depth=18)
        >>> self.eval()
        >>> inputs = torch.rand(1, 3, 32, 32)
        >>> level_outputs = self.forward(inputs)
        >>> for level_out in level_outputs:
        ...     print(tuple(level_out.shape))
        (1, 64, 8, 8)
        (1, 128, 4, 4)
        (1, 256, 2, 2)
        (1, 512, 1, 1)
    """
    arch_settings = {
        18: (BasicBlock, (2, 2, 2, 2)),
        34: (BasicBlock, (3, 4, 6, 3)),
        50: (Bottleneck, (3, 4, 6, 3)),
        101: (Bottleneck, (3, 4, 23, 3)),
        152: (Bottleneck, (3, 8, 36, 3))
    }

    def __init__(self,
                 depth,
                 in_channels=3,
                 stem_channels=64,
                 base_channels=64,
                 num_stages=4,
                 strides=(1, 2, 2, 2),
                 dilations=(1, 1, 1, 1),
                 out_indices=(0, 1, 2, 3),
                 style='pytorch',
                 deep_stem=False,
                 avg_down=False,
                 frozen_stages=-1,
                 conv_cfg=None,
                 norm_cfg=dict(type='BN', requires_grad=True),
                 norm_eval=True,
                 dcn=None,
                 stage_with_dcn=(False, False, False, False),
                 plugins=None,
                 with_cp=False,
                 zero_init_residual=True):
        super(ResNet, self).__init__()
        # ========================== 初始化属性 =============================
        if depth not in self.arch_settings:
            raise KeyError(f'invalid depth {depth} for resnet')
        self.depth = depth
        self.stem_channels = stem_channels
        self.base_channels = base_channels
        self.num_stages = num_stages
        assert num_stages >= 1 and num_stages <= 4
        self.strides = strides
        self.dilations = dilations
        assert len(strides) == len(dilations) == num_stages
        self.out_indices = out_indices
        assert max(out_indices) < num_stages
        self.style = style
        self.deep_stem = deep_stem
        self.avg_down = avg_down
        self.frozen_stages = frozen_stages
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg
        self.with_cp = with_cp
        self.norm_eval = norm_eval
        self.dcn = dcn
        self.stage_with_dcn = stage_with_dcn
        if dcn is not None:
            assert len(stage_with_dcn) == num_stages
        self.plugins = plugins
        self.zero_init_residual = zero_init_residual
        # ===================================================================
        self.block, stage_blocks = self.arch_settings[depth]
        self.stage_blocks = stage_blocks[:num_stages]

        # stem 层
        self.inplanes = stem_channels
        self._make_stem_layer(in_channels, stem_channels)

        # res 层
        self.res_layers = []
        for i, num_blocks in enumerate(self.stage_blocks):
            stride = strides[i]
            dilation = dilations[i]
            dcn = self.dcn if self.stage_with_dcn[i] else None
            if plugins is not None:
                stage_plugins = self.make_stage_plugins(plugins, i)
            else:
                stage_plugins = None
            planes = base_channels * 2**i
            res_layer = self.make_res_layer(
                block=self.block,
                inplanes=self.inplanes,
                planes=planes,
                num_blocks=num_blocks,
                stride=stride,
                dilation=dilation,
                style=self.style,
                avg_down=self.avg_down,
                with_cp=with_cp,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                dcn=dcn,
                plugins=stage_plugins)
            self.inplanes = planes * self.block.expansion
            layer_name = f'layer{i + 1}'
            self.add_module(layer_name, res_layer)
            self.res_layers.append(layer_name)

        self._freeze_stages()

        self.feat_dim = self.block.expansion * base_channels * 2**(
            len(self.stage_blocks) - 1)

    def make_stage_plugins(self, plugins, stage_idx):
        """Make plugins for ResNet ``stage_idx`` th stage.

        Currently we support to insert ``context_block``,
        ``empirical_attention_block``, ``nonlocal_block`` into the backbone
        like ResNet/ResNeXt. They could be inserted after conv1/conv2/conv3 of
        Bottleneck.

        An example of plugins format could be:

        Examples:
            >>> plugins=[
            ...     dict(cfg=dict(type='xxx', arg1='xxx'),
            ...          stages=(False, True, True, True),
            ...          position='after_conv2'),
            ...     dict(cfg=dict(type='yyy'),
            ...          stages=(True, True, True, True),
            ...          position='after_conv3'),
            ...     dict(cfg=dict(type='zzz', postfix='1'),
            ...          stages=(True, True, True, True),
            ...          position='after_conv3'),
            ...     dict(cfg=dict(type='zzz', postfix='2'),
            ...          stages=(True, True, True, True),
            ...          position='after_conv3')
            ... ]
            >>> self = ResNet(depth=18)
            >>> stage_plugins = self.make_stage_plugins(plugins, 0)
            >>> assert len(stage_plugins) == 3

        Suppose ``stage_idx=0``, the structure of blocks in the stage would be:

        .. code-block:: none

            conv1-> conv2->conv3->yyy->zzz1->zzz2

        Suppose 'stage_idx=1', the structure of blocks in the stage would be:

        .. code-block:: none

            conv1-> conv2->xxx->conv3->yyy->zzz1->zzz2

        If stages is missing, the plugin would be applied to all stages.

        Args:
            plugins (list[dict]): List of plugins cfg to build. The postfix is
                required if multiple same type plugins are inserted.
            stage_idx (int): Index of stage to build

        Returns:
            list[dict]: Plugins for current stage
        """
        stage_plugins = []
        for plugin in plugins:
            plugin = plugin.copy()
            stages = plugin.pop('stages', None)
            assert stages is None or len(stages) == self.num_stages
            # whether to insert plugin into current stage
            if stages is None or stages[stage_idx]:
                stage_plugins.append(plugin)

        return stage_plugins

    def make_res_layer(self, **kwargs):
        """Pack all blocks in a stage into a ``ResLayer``."""
        return ResLayer(**kwargs)

    @property
    def norm1(self):
        """nn.Module: the normalization layer named "norm1" """
        return getattr(self, self.norm1_name)

     def _make_stem_layer(self, in_channels, stem_channels):
        # 使用 deep stem, 即 3 个 3x3 conv.
        if self.deep_stem:
            self.stem = nn.Sequential(
                build_conv_layer(
                    self.conv_cfg,
                    in_channels,
                    stem_channels // 2,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    bias=False),
                build_norm_layer(self.norm_cfg, stem_channels // 2)[1],
                nn.ReLU(inplace=True),
                build_conv_layer(
                    self.conv_cfg,
                    stem_channels // 2,
                    stem_channels // 2,
                    kernel_size=3,
                    stride=1,
                    padding=1,
                    bias=False),
                build_norm_layer(self.norm_cfg, stem_channels // 2)[1],
                nn.ReLU(inplace=True),
                build_conv_layer(
                    self.conv_cfg,
                    stem_channels // 2,
                    stem_channels,
                    kernel_size=3,
                    stride=1,
                    padding=1,
                    bias=False),
                build_norm_layer(self.norm_cfg, stem_channels)[1],
                nn.ReLU(inplace=True))
        # 使用原版的 stem 即 7x7 conv
        else:
            self.conv1 = build_conv_layer(
                self.conv_cfg,
                in_channels,
                stem_channels,
                kernel_size=7,
                stride=2,
                padding=3,
                bias=False)
            self.norm1_name, norm1 = build_norm_layer(
                self.norm_cfg, stem_channels, postfix=1)
            self.add_module(self.norm1_name, norm1)
            self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def _freeze_stages(self):
        # 冻结 stem 层 --> stage 为 0
        if self.frozen_stages >= 0:
            if self.deep_stem:
                self.stem.eval()
                for param in self.stem.parameters():
                    param.requires_grad = False
            else:
                self.norm1.eval()
                for m in [self.conv1, self.norm1]:
                    for param in m.parameters():
                        param.requires_grad = False

        # 冻结 res 层  --> stage 大于 0
        for i in range(1, self.frozen_stages + 1):
            m = getattr(self, f'layer{i}')
            m.eval()
            for param in m.parameters():
                param.requires_grad = False

    def init_weights(self, pretrained=None):
        """Initialize the weights in backbone.

        Args:
            pretrained (str, optional): Path to pre-trained weights.
                Defaults to None.
        """
        if isinstance(pretrained, str):
            logger = get_root_logger()
            load_checkpoint(self, pretrained, strict=False, logger=logger)
        elif pretrained is None:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    kaiming_init(m)
                elif isinstance(m, (_BatchNorm, nn.GroupNorm)):
                    constant_init(m, 1)

            if self.dcn is not None:
                for m in self.modules():
                    if isinstance(m, Bottleneck) and hasattr(
                            m.conv2, 'conv_offset'):
                        constant_init(m.conv2.conv_offset, 0)

            if self.zero_init_residual:
                for m in self.modules():
                    if isinstance(m, Bottleneck):
                        constant_init(m.norm3, 0)
                    elif isinstance(m, BasicBlock):
                        constant_init(m.norm2, 0)
        else:
            raise TypeError('pretrained must be a str or None')

    def forward(self, x):
        """Forward function."""
        if self.deep_stem:
            x = self.stem(x)
        else:
            x = self.conv1(x)
            x = self.norm1(x)
            x = self.relu(x)
        x = self.maxpool(x)
        outs = []
        for i, layer_name in enumerate(self.res_layers):
            res_layer = getattr(self, layer_name)
            x = res_layer(x)
            if i in self.out_indices:
                outs.append(x)
        return tuple(outs)

    def train(self, mode=True):
        """Convert the model into training mode while keep normalization layer
        freezed."""
        super(ResNet, self).train(mode)
        self._freeze_stages()
        if mode and self.norm_eval:
            for m in self.modules():
                # trick: eval have effect on BatchNorm only
                if isinstance(m, _BatchNorm):
                    m.eval()

[更多ResNeXt](https://zhuanlan.zhihu.com/p/166248079)

# [MMDetection Faster R-CNN 源码详解（二）](https://zhuanlan.zhihu.com/p/183098688)

本篇文章中，会重点剖析 MMDetection 实现的 Faster R-CNN 中的 neck 相关的源码。

## 一、FPN 的思想(Feature Pyramid Network)

在 MMDetection 中，Faster R-CNN 的 baseline 结合了 FPN，通过 FPN 会大大的提升检测的效果。为什么 FPN 有这么强的作用呢？我们从如下两点分析

1. 提取了多尺度的特征: 相对于单尺度的特征，多尺度的特征对大中小物体都能覆盖。对于 CNN 网络，浅层特征的特征途较大, 感受野较小, 方便检测小物体. 深层特征的特征图较小, 感受野较大, 方便检测大物体. 所以使用多尺度的特征对于不同大小的物体都有很好的覆盖效果。

2. 融合了各个尺度的特征：对于浅层的特征图，对位置敏感但是语义信息较弱。深层的特征图，对位置不敏感但是语义信息较强。那么我们就可以将当前的尺度与更深层的尺度的特征图相融合。这样既有深层的语义信息又对位置敏感。


## 二、Faster R-CNN 结合 FPN
接下来我们就来看一下如何将 Faster R-CNN 与 FPN 向结合，下图是将 Faster R-CNN 与 FPN 融合的网络结构。

![](https://pic1.zhimg.com/80/v2-4aee99e6842420bf682433ff4c0ff720_720w.jpg) Faster R-CNN 结合 FPN 的网络结构


拿到 backbone 的后四个尺度的输出，也就是 C2 ～ C5。以 resnet 50 为例，通道数分别为：256、512、1024、2048。先使用 1×1 的卷积，将 resnet 输出的特征图压缩通道数到 256，得到 P2 ～ P5。然后将浅层的特征图与深层的特征图相加（需要先上采样，保证特征图大小相同再相加）。再使用 3 × 3 的卷积核进行卷积，此步骤会将相加的特征进一步融合，得到 FPN 输出的特征，也就是 P2 ～ P5。在 P5 上使用 1 × 1 步长为 2 的 max pool 进行下采样，得到 P6。所以当数据经过 FPN 后，我们会拿到五个尺度的特征，也就是 P2 ～ P6。每个特征的通道数都是 256。因为每个尺度输出的特征图的通道数固定，所以我们就可以使用相同的头部进行预测了。

![](https://pic1.zhimg.com/80/v2-c0172be282021a1029f7b72b51079ffe_1440w.jpg)


![](https://pic2.zhimg.com/v2-e49ebcf931b5cf424ed311338f9ff35d_b.jpg)


## 三、RetinaNet 中的 FPN

![](https://pic1.zhimg.com/80/v2-b5a2faa28bd62f532d4fb405159f15dc_720w.jpg)

在 RetinaNet 中，利用的 backbone 的特征是 C3 ～ C5。将 C3 ～ C5 的特征图经过 1×1 的卷积压缩通道数，再将深层特征图与浅层特征图相加。然后使用 3 × 3 的卷积进一步融合特征。这样操作后，我们提取了从 P3 ～ P5 的特征。对于 C5，我们用 3 × 3 步长为 2，padding 为 1 的卷积进行两次下采样就得到 FPN 的 P6 和 P7。



## 四、FPN 源码分析
（一）Faster R-CNN 的 FPN 配置
紧接着，将x输入到neck中,也就是fpn结构中，我这里fpn的设置参数为

In [ ]:
neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),

（二）RetinaNet 的 FPN 配置

In [ ]:
neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        start_level=1,
        add_extra_convs='on_input',
        num_outs=5),

结合上面的配置文件我们来看一下源码：


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from mmcv.cnn import ConvModule, xavier_init

from mmdet.core import auto_fp16
from ..builder import NECKS

@NECKS.register_module()
class FPN(nn.Module):
    """Feature Pyramid Network.  (https://arxiv.org/abs/1612.03144)

    Args:
        in_channels:                    (List[int])     每个尺度的输入通道数, 也是 backbone 的输出通道数.
        out_channels:                  (int)     fpn 的输出通道数, 所有尺度的输出通道数相同, 都是一个值.
        num_outs:                      (int)     输出 stage 的个数.(可以附加额外的层, num_outs 不一定等于 in_channels)
        start_level:                   (int)     使用 backbone 的起始 stage 索引, 默认为 0.
        end_level:                     (int)     使用 backbone 的终止 stage 索引。
                                                默认为 -1, 代表到最后一层(包括)全使用.
        add_extra_convs:        (bool | str)     可以是 bool 或 str:
                                                (bool)  bool 代表是否添加额外的层.(默认值: False)
                                                        True:   在最顶层 feature map 上添加额外的卷积层,
                                                                具体的模式需要 extra_convs_on_inputs 指定.
                                                        False:  不添加额外的卷积层
                                                (str)   str  需要指定 extra convs 的输入的 feature map 的来源
                                                        'on_input':     最高层的 feature map 作为 extra 的输入
                                                        'on_lateral':   最高层的 lateral 结果 作为 extra 的输入
                                                        'on_output':    最高层的经过 conv 的 lateral 结果作为 extra 的输入
        extra_convs_on_inputs:  (bool, deprecated)  True  等同于 `add_extra_convs='on_input'
                                                    False 等同于 `add_extra_convs='on_output'
                                                    默认值为True
        relu_before_extra_convs:      (bool)     是否在 extra conv 前使用 relu. (默认值: False)
        no_norm_on_lateral:           (bool)     是否对 lateral 使用 bn. (默认值: False)
        conv_cfg:                     (dict)     构建 conv 层的 config 字典. (默认值: None)
        norm_cfg:                     (dict)     构建  bn  层的 config 字典. (默认值: None)
        act_cfg:                      (dict)     构建 activation  层的 config 字典. (默认值: None)
        upsample_cfg:                 (dict)     构建 interpolate 层的 config 字典. (默认值: `dict(mode='nearest')`)                                                    
        
        Example:
        >>> import torch
        >>> in_channels = [2, 3, 5, 7]
        >>> scales = [340, 170, 84, 43]
        >>> inputs = [torch.rand(1, c, s, s)
        ...           for c, s in zip(in_channels, scales)]
        >>> self = FPN(in_channels, 11, len(in_channels)).eval()
        >>> outputs = self.forward(inputs)
        >>> for i in range(len(outputs)):
        ...     print(f'outputs[{i}].shape = {outputs[i].shape}')
        outputs[0].shape = torch.Size([1, 11, 340, 340])
        outputs[1].shape = torch.Size([1, 11, 170, 170])
        outputs[2].shape = torch.Size([1, 11, 84, 84])
        outputs[3].shape = torch.Size([1, 11, 43, 43])
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 num_outs,
                 start_level=0,
                 end_level=-1,
                 add_extra_convs=False,
                 extra_convs_on_inputs=True,
                 relu_before_extra_convs=False,
                 no_norm_on_lateral=False,
                 conv_cfg=None,
                 norm_cfg=None,
                 act_cfg=None,
                 upsample_cfg=dict(mode='nearest')):
        super(FPN, self).__init__()
        assert isinstance(in_channels, list)
        self.in_channels = in_channels          # [256, 512, 1024, 2048]
        self.out_channels = out_channels        # 256
        self.num_ins = len(in_channels)         # 4
        self.num_outs = num_outs                # 5
        self.relu_before_extra_convs = relu_before_extra_convs  # False
        self.no_norm_on_lateral = no_norm_on_lateral            # False
        self.fp16_enabled = False
        self.upsample_cfg = upsample_cfg.copy()

        # end_level 是对 backbone 输出的尺度中使用的最后一个尺度的索引
        # 如果是 -1 表示使用 backbone 最后一个 feature map, 作为最终的索引.
        if end_level == -1:
            self.backbone_end_level = self.num_ins      #4
            # 因为还有 extra conv 所以存在 num_outs > num_ins - start_level 的情况
            assert num_outs >= self.num_ins - start_level
        else:
            # 如果 end_level < inputs, 说明不使用 backbone 全部的尺度, 并且不会提供额外的层.
            self.backbone_end_level = end_level
            assert end_level <= len(in_channels)
            assert num_outs == end_level - start_level

        self.start_level = start_level                      # 0
        self.end_level = end_level                          # -1
        self.add_extra_convs = add_extra_convs              # False
        assert isinstance(add_extra_convs, (str, bool))
        # add_extra_convs 可以是 bool 或 str
        # 1. add_extra_convs 是 str
        if isinstance(add_extra_convs, str):
            # 确保 add_extra_convs 是 'on_input', 'on_lateral' 或 'on_output'
            assert add_extra_convs in ('on_input', 'on_lateral', 'on_output')
        # 2. add_extra_convs 是 bool, 需要看 extra_convs_on_inputs
        elif add_extra_convs:
            if extra_convs_on_inputs:
                # For compatibility with previous release
                # TODO: deprecate `extra_convs_on_inputs`
                self.add_extra_convs = 'on_input'
            else:
                self.add_extra_convs = 'on_output'

        self.lateral_convs = nn.ModuleList()
        self.fpn_convs = nn.ModuleList()

        # 构建Lateral conv 和 fpn conv 
        for i in range(self.start_level, self.backbone_end_level):
            # 水平卷积(lateral conv): 1×1, C=256,
            l_conv = ConvModule(
                in_channels[i],
                out_channels,
                1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg if not self.no_norm_on_lateral else None,
                act_cfg=act_cfg,
                inplace=False)
            # fpn 输出卷积: 3×3, C=256, P=1
            fpn_conv = ConvModule(
                out_channels,
                out_channels,
                3,
                padding=1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                act_cfg=act_cfg,
                inplace=False)
            
            self.lateral_convs.append(l_conv)
            self.fpn_convs.append(fpn_conv)

        # add extra conv layers (e.g., RetinaNet)
        extra_levels = num_outs - self.backbone_end_level + self.start_level
        # 只有 add_extra_convs 为 True 或 str 时才添加 extra_convs
        if self.add_extra_convs and extra_levels >= 1:
            for i in range(extra_levels):
                if i == 0 and self.add_extra_convs == 'on_input':
                    in_channels = self.in_channels[self.backbone_end_level - 1]
                else:
                    in_channels = out_channels
                # extra conv 是3x3步长为2, padding为1的卷积
                extra_fpn_conv = ConvModule(
                    in_channels,
                    out_channels,
                    3,
                    stride=2,
                    padding=1,
                    conv_cfg=conv_cfg,
                    norm_cfg=norm_cfg,
                    act_cfg=act_cfg,
                    inplace=False)
                self.fpn_convs.append(extra_fpn_conv)
    # default init_weights for conv(msra) and norm in ConvModule
    def init_weights(self):
        """Initialize the weights of FPN module."""
        # 使用xavier初始化卷积层
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                xavier_init(m, distribution='uniform')
    
    @auto_fp16()
    def forward(self, inputs):
        """Forward function."""
        assert len(inputs) == len(self.in_channels)

        # ====================== 进行水平计算(1x1卷积) ====================
        laterals = [
            lateral_conv(inputs[i + self.start_level])
            for i, lateral_conv in enumerate(self.lateral_convs)
        ]
        # ==============================================================

        # ========================== 计算 top-down =============================
        used_backbone_levels = len(laterals)
        # 自上至下将 laterals 里面的结果更新为经过 top-down 的结果.
        for i in range(used_backbone_levels - 1, 0, -1):
           # In some cases, fixing `scale factor` (e.g. 2) is preferred, but
            #  it cannot co-exist with `size` in `F.interpolate`.
            # 有 scale 的情况
            if 'scale_factor' in self.upsample_cfg:
                 # 因为range函数不包括右边的端点, 所以可以使用 i - 1
                 laterals[i-1] += F.interpolate(laterals[i], **self.upsample_cfg)
            # 没有 scale 的情况, 需要计算下层的 feature map 大小.
            else:
                 # 计算下层 feature map 大小
                prev_shape = laterals[i - 1].shape[2:]
                laterals[i - 1] += F.interpolate(
                    laterals[i], size=prev_shape, **self.upsample_cfg)
        # =====================================================================

        # ========================== 计算输出的结果 =============================
        # part 1: 计算所有 lateral 的输出的结果
        outs = [
            self.fpn_convs[i](laterals[i]) for i in range(used_backbone_levels)
        ]
        
        # part 2: 添加 extra levels
        if self.num_outs > len(outs):
            # 使用 max pool 获得更高层的输出信息, 如: Faster R-CNN, Mask R-CNN (4 lateral + 1 max pool)
            if not self.add_extra_convs:
                for i in range(self.num_outs - used_backbone_levels):
                    outs.append(F.max_pool2d(outs[-1], 1, stride=2))
            # 添加额外的卷积层获得高层输出信息, 如: RetinaNet (3 lateral + 2 conv3x3 stride2)
            else:
                # 'on_input':   最高层的 feature map 作为 extra 的输入
                if self.add_extra_convs == 'on_input':
                    extra_source = inputs[self.backbone_end_level - 1]
                # 'on_lateral': 最高层的 lateral 结果 作为 extra 的输入
                elif self.add_extra_convs == 'on_lateral':
                    extra_source = laterals[-1]
                # 'on_output':  最高层的经过 conv 的 lateral 结果作为 extra 的输入
                elif self.add_extra_convs == 'on_output':
                    extra_source = outs[-1]
                else:
                    raise NotImplementedError
                # 计算 input extra
                outs.append(self.fpn_convs[used_backbone_levels](extra_source))
                # 计算 extra
                for i in range(used_backbone_levels + 1, self.num_outs):
                    if self.relu_before_extra_convs:
                        outs.append(self.fpn_convs[i](F.relu(outs[-1])))
                    else:
                        outs.append(self.fpn_convs[i](outs[-1]))
        return tuple(outs)


# [MMDetection Faster R-CNN 源码详解（三)](https://zhuanlan.zhihu.com/p/184618997)



# [MMDetection Faster R-CNN 源码详解（四)](https://zhuanlan.zhihu.com/p/194285023)

RPN（源码篇）
1. 整体结构
2. BaseDenseHead
4. RPNTestMixin
5. RPNHead


# [源码解读：Faster RCNN的细节（一）](https://zhuanlan.zhihu.com/p/65471961)

![](https://pica.zhimg.com/v2-2a51ac64bbb4b620ab1aa90f1793b898_1440w.jpg?source=172ae18b)